### This data is for Copy of 2021_Jan_Utilisation_Based_Monthly_EAP_CS_v4.xlsx
***

Import libraries:

In [1]:
import pandas as pd
import pyodbc as p

In [2]:
def limit10_fn(conn,db_name, table_name):
    query="""select * from {db_name}.{table_name} limit 10""".format(db_name=db_name,table_name=table_name)
    return pd.read_sql(query,conn)

This is the raw data script:

In [3]:
sql_query="""select * from 
(
with cte as (
select distinct
ben.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num 
,ben.dep_fk 
,ben.comp_name
,mipst_dbo.tsf_ccm_template.template_label	
,mipst_dbo.tsf_ccm_template.template_description
,user_code
,replace(substring(call_start_datetime,1,10), '-','') as call_date 
,coalesce(g.comp_name,'none') as group_name	
,dep_type
,ben.comp_fk
,scheme_fk

FROM mipst_dbo.tsf_ccm_template
left join mipst_dbo.tsf_cct_call call ON call.template_obj = mipst_dbo.tsf_ccm_template.template_obj and call.status_key = 'cc_StCctcaComp' 
inner join mipst_dbo.tsd_cc_relationship real ON real.owning_obj::NUMERIC = call.call_obj
and UPPER(owning_entity_mnemonic) = 'CCTCA'
AND   UPPER(related_entity_mnemonic) in ('MEMBER','MEMDEP')
inner join  mipbi_dbo.td_beneficiary ben ON ben.ms_pk = coalesce(nullif(split_part(related_key, '|',1), ''), '0') --and scheme_fk='126'
and dep_fk::int	=coalesce(nullif(split_part(related_key, '|',2), ''), '0')::int
left join mipst_dbo.tsd_ben_memcom g  on  (case when ben.group_code='0' then ben.comp_fk else ben.group_code end)=g.comp_fk
) select cte.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num as identity_number 
,dep_fk as dependent_no
,dep_type
,cte.comp_name as companyname
, group_name
,cte.template_label	
,cte.template_description
,cte.user_code
, cte.call_date::varchar::date as call_date 
, array_to_string(array_agg(distinct (coalesce((c.Problem||'-'||c.sub_category),'') ||','||coalesce(l.Problem ,'')||','||coalesce(f.Problem ,''))),',') as Problem_name
from cte 
INNER JOIN 
mipst_dbo.tsd_dates on mipst_dbo.tsd_dates.date_pk::VARCHAR =call_date 
left join 
mipst_dbo.legal_raw_2021 l on cte.ms_pk=l.ms_pk and cte.dep_fk=l.dependent_no::int and  cte.call_date::varchar::date =l.call_date and cte.template_label=l.template_label and cte.comp_name=l.comp_name 
				  and  cte.scheme_fk=l.scheme_fk and cte.user_code=l.user_code  left join
mipst_dbo.councelling_raw_2021 c on cte.ms_pk=c.ms_pk and cte.dep_fk=c.dependent_no::int and  cte.call_date::varchar::date =c.call_date and cte.template_label=c.template_label and cte.comp_name=c.comp_name 			 
and  cte.scheme_fk=c.scheme_fk and cte.user_code=c.user_code  left join
mipst_dbo.financial_raw_2021 f on cte.ms_pk=f.ms_pk and cte.dep_fk=f.dependent_no::int and  cte.call_date::varchar::date =f.call_date and cte.template_label=f.template_label and cte.comp_name=f.comp_name 			 
and  cte.scheme_fk=f.scheme_fk and cte.user_code=f.user_code  
group by cte.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num  
,dep_fk
,dep_type				  
,cte.comp_name 
, group_name
,cte.template_label	
,cte.template_description
,cte.user_code
, cte.call_date::varchar::date 
) a"""

Here we connect to the __kx_bi__ postgres database and execute the above query to generate the EAP raw data set:

In [4]:
conn_str = (
    "DRIVER={PostgreSQL ODBC Driver(UNICODE)};"
    "DATABASE=kx_bi;"
    "UID=postgres;"
    "PWD=postgres;"
    "SERVER=172.31.244.44;"
    "PORT=5432;"
    )

conn = p.connect(conn_str)
# cursor = conn.cursor()

eapraw_df = pd.read_sql(sql_query,conn)

OperationalError: ('08001', '[08001] could not connect to server: Connection timed out (0x0000274C/10060)\n\tIs the server running on host "172.31.244.44" and accepting\n\tTCP/IP connections on port 5432?\n (101) (SQLDriverConnect)')

In [5]:
eapraw_df[eapraw_df.companyname=='Mimecast South Africa (Pty) Ltd']

,ms_pk,scheme_name,first_name,surname,gender,birth_date,main_email,main_work_tel,identity_number,dependent_no,dep_type,companyname,group_name,template_label,template_description,user_code,call_date,problem_name
2862,01677203,KAELO LIFESTYLE,Palesa,Phomolo,F,19921220.0,None,None,9212200129080,0,M,Mimecast South Africa (Pty) Ltd,Mimecast South Africa (Pty) Ltd,Case Note,EAP Case Note,karabom,2020-06-03,",,"
2863,01677203,KAELO LIFESTYLE,Palesa,Phomolo,F,19921220.0,None,None,9212200129080,0,M,Mimecast South Africa (Pty) Ltd,Mimecast South Africa (Pty) Ltd,Case Note,EAP Case Note,lucianoa,2020-01-24,",,"
2864,01677203,KAELO LIFESTYLE,Palesa,Phomolo,F,19921220.0,None,None,9212200129080,0,M,Mimecast South Africa (Pty) Ltd,Mimecast South Africa (Pty) Ltd,Case Note,EAP Case Note,thabilem,2020-06-03,",,"
2865,01677203,KAELO LIFESTYLE,Palesa,Phomolo,F,19921220.0,None,None,9212200129080,0,M,Mimecast South Africa (Pty) Ltd,Mimecast South Africa (Pty) Ltd,Closed Case Note,Closed Case Note,pollyt,2020-12-01,",,"
2866,01677203,KAELO LIFESTYLE,Palesa,Phomolo,F,19921220.0,None,None,9212200129080,0,M,Mimecast South Africa (Pty) Ltd,Mimecast South Africa (Pty) Ltd,EAP Unsuccessful communication,EAP Unsuccessful communication,lucianoa,2020-01-20,",,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93721,518807,KAELO LIFESTYLE,VERONICA,TAKALANI,F,19920509.0,None,None,9205090808086,0,M,Mimecast South Africa (Pty) Ltd,Mimecast South Africa (Pty) Ltd,Telephonic Counselling Form,Telephonic Counselling Form,lingeshverin,2020-12-21,"Bereavement-loss of mum,,,Organisational Chall..."
93742,518936,KAELO LIFESTYLE,ZUBEIDA,WILLIAMS,F,19840331.0,None,None,8403310095086,0,M,Mimecast South Africa (Pty) Ltd,Mimecast South Africa (Pty) Ltd,Telephonic Counselling Form,Telephonic Counselling Form,PRINCEMA,2020-12-18,"Psychiatric-Depression,,"
93858,519317,KAELO LIFESTYLE,NOBUHLE,PHEHLUKWAYO,F,19920905.0,None,None,9209050722089,0,M,Mimecast South Africa (Pty) Ltd,Mimecast South Africa (Pty) Ltd,Legal Advice,Legal Advice,lingeshverin,2021-01-08,",Consumer protection issue,"
94195,520520,KAELO LIFESTYLE,NIKIWE,NHLAPHO,F,19900820.0,None,None,9008200293086,0,M,Mimecast South Africa (Pty) Ltd,Mimecast South Africa (Pty) Ltd,Telephonic Counselling Form,Telephonic Counselling Form,senatel,2021-02-12,"Organisational Challenges-General Discontent,,..."


In [6]:
eapraw_df.iloc[3366]["problem_name"]
# [eapraw_df.ms_pk=="""01678068"""]

'Trauma-Other,,'

In [7]:
eapraw_df.iloc[3366]["problem_name"]

'Trauma-Other,,'

In [8]:
list(eapraw_df)

['ms_pk',
 'scheme_name',
 'first_name',
 'surname',
 'gender',
 'birth_date',
 'main_email',
 'main_work_tel',
 'identity_number',
 'dependent_no',
 'dep_type',
 'companyname',
 'group_name',
 'template_label',
 'template_description',
 'user_code',
 'call_date',
 'problem_name']

In [10]:
"""
mipst_dbo.tsf_ccm_template
mipst_dbo.tsf_cct_call
mipst_dbo.tsd_cc_relationship
mipbi_dbo.td_beneficiary
mipst_dbo.tsd_ben_memcom
mipst_dbo.tsd_dates
mipst_dbo.legal_raw_2021
mipst_dbo.councelling_raw_2021
mipst_dbo.financial_raw_2021
"""

'\nmipst_dbo.tsf_ccm_template\nmipst_dbo.tsf_cct_call\nmipst_dbo.tsd_cc_relationship\nmipbi_dbo.td_beneficiary\nmipst_dbo.tsd_ben_memcom\nmipst_dbo.tsd_dates\nmipst_dbo.legal_raw_2021\nmipst_dbo.councelling_raw_2021\nmipst_dbo.financial_raw_2021\n'

In [11]:
tsf_ccm_template_df=limit10_fn(conn=conn,db_name="mipst_dbo", table_name="tsf_ccm_template")
tsf_cct_call_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="tsf_cct_call")
tsd_cc_relationship_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="tsd_cc_relationship")
td_beneficiary_df=limit10_fn(conn=conn,db_name="mipbi_dbo",table_name="td_beneficiary")
tsd_ben_memcom_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="tsd_ben_memcom")
tsd_dates_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="tsd_dates")
legal_raw_2021_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="legal_raw_2021")
councelling_raw_2021_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="councelling_raw_2021")
financial_raw_2021_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="financial_raw_2021")

In [12]:
# tsf_ccm_template_df
list(tsf_ccm_template_df)

['template_obj',
 'status_key',
 'category_key',
 'object_obj',
 'template_code',
 'template_label',
 'template_description',
 'template_field_namelist',
 'template_field_attriblist',
 'last_modified_datetime']

In [13]:
list(tsf_cct_call_df)

['call_obj',
 'template_obj',
 'status_key',
 'call_reference',
 'user_code',
 'direction_acronym_key',
 'method_acronym_key',
 'abort_reason_acronym_key',
 'call_field_namelist',
 'call_field_attriblist',
 'call_caller_name',
 'call_telephone_number',
 'call_start_datetime',
 'call_end_datetime',
 'call_enquiry',
 'call_response',
 'call_wordindex',
 'last_modified_datetime',
 'call_start_datetime_int']

In [14]:
list(tsd_cc_relationship_df)

['relationship_obj',
 'relation_key',
 'owning_entity_mnemonic',
 'owning_obj',
 'owning_key',
 'related_entity_mnemonic',
 'related_obj',
 'related_key']

In [15]:
list(td_beneficiary_df)

['admin_fk',
 'scheme_fk',
 'scheme_name',
 'ms_pk',
 'mem_status',
 'dep_fk',
 'dep_type',
 'surname',
 'initials',
 'birth_date',
 'gender',
 'race_code',
 'phys1',
 'phys2',
 'phys3',
 'phys4',
 'phys5',
 'phys_code',
 'post1',
 'post2',
 'post3',
 'post4',
 'post5',
 'post_code',
 'start_date',
 'end_date',
 'join_date',
 'resign_date',
 'benefit_date',
 'suspend_date',
 'income_amount',
 'broker_fk',
 'comp_fk',
 'comp_name',
 'load_date',
 'sub_company_fk',
 'subcomp_name',
 'income_cat',
 'resign_code',
 'resign_reason',
 'main_type',
 'relationship',
 'title',
 'main_tel',
 'main_cell',
 'main_work_tel',
 'main_email',
 'comp_post_code',
 'inact_start_1',
 'inact_start_2',
 'inact_end_1',
 'inact_end_2',
 'wait_per_cat',
 'income_proof_date',
 'graduation_date',
 'underwrite_cat',
 'first_name',
 'group_class',
 'group_code',
 'id_num',
 'pref_lang',
 'home_lang',
 'penalty_percentage',
 'orig_joined',
 'main_dep',
 'suspend_type',
 'act_status',
 'nick_name',
 'student_number'

In [16]:
list(tsd_ben_memcom_df)

['comp_fk',
 'comp_name',
 'cre_cont_code',
 'group_class',
 'group_code',
 'cont_person',
 'tel']

In [17]:
list(tsd_dates_df)

['year_number',
 'month_number',
 'day_of_year_number',
 'day_of_month_number',
 'day_of_week_number',
 'week_of_year_number',
 'day_name',
 'month_name',
 'quarter_number',
 'quarter_name',
 'year_quarter_name',
 'weekend_ind',
 'days_in_month_qty',
 'date_pk',
 'day_desc',
 'week_sk',
 'day_date',
 'week_name',
 'week_of_month_number',
 'week_of_month_name',
 'month_sk',
 'quarter_sk',
 'year_sk',
 'year_sort_number',
 'day_of_week_sort_name']

In [18]:
list(legal_raw_2021_df)

['ms_pk',
 'comp_name',
 'comp_fk',
 'user_code',
 'template_label',
 'call_date',
 'dependent_no',
 'problem',
 'sub_category',
 'scheme_fk']

In [19]:
list(councelling_raw_2021_df)

['ms_pk',
 'comp_name',
 'comp_fk',
 'user_code',
 'template_label',
 'call_date',
 'dependent_no',
 'problem',
 'sub_category',
 'scheme_fk']

In [20]:
list(financial_raw_2021_df)

['ms_pk',
 'comp_name',
 'comp_fk',
 'user_code',
 'template_label',
 'call_date',
 'dependent_no',
 'problem',
 'sub_category',
 'scheme_fk']

In [21]:
list(councelling_raw_2021_df)

['ms_pk',
 'comp_name',
 'comp_fk',
 'user_code',
 'template_label',
 'call_date',
 'dependent_no',
 'problem',
 'sub_category',
 'scheme_fk']

In [22]:
councelling_raw_2021_df

,ms_pk,comp_name,comp_fk,user_code,template_label,call_date,dependent_no,problem,sub_category,scheme_fk
0,01690797,Jackpersad & Partners,600404,masetshegob,Offsite Face to Face Counselling Form,2020-05-05,0,Stress,Eposodic Stress,126
1,01736993,AVBOB_Head Office,600509,thabilem,Telephonic Counselling Form,2020-12-17,0,Information,Service Enquiry,126
2,1621680,Ster-Kinekor Theatres (Pty) Ltd,600169,fikilen,Telephonic Counselling Form,2020-02-28,0,Stress,Acute Stress,126
3,1531371,Capitec_BSC_Western Cape,600206,pollyt,Telephonic Counselling Form,2020-06-17,0,Suicide,Contemplated,126
4,513109,WNS Global Services,100990,thabilem,Telephonic Counselling Form,2020-05-11,0,Stress,Acute Stress,108
5,1600366,Tau Phola (Pty) Ltd,600147,yvetter,Telephonic Counselling Form,2019-03-12,0,Organisational Challenges,Work conflict,126
6,1501647,Saint-Gobain Samrand Delegation,100979,karabom,Telephonic Counselling Form,2019-04-15,0,Stress,Acute Stress,126
7,1540695,Enablement,600254,lingeshverin,Telephonic Counselling Form,2019-05-21,0,Information,Service Enquiry,126
8,1518381,Capitec_Campus_Western Cape,600224,anns,Telephonic Counselling Form,2020-11-27,0,Psychiatric,Anxiety,126
9,514558,Top T,600231,thabilem,Telephonic Counselling Form,2020-05-12,0,Bereavement,Loss of loved one,126


In [23]:
conn_str = (
    "DRIVER={PostgreSQL ODBC Driver(UNICODE)};"
    "DATABASE=kx_bi;"
    "UID=postgres;"
    "PWD=postgres;"
    "SERVER=172.31.244.44;"
    "PORT=5432;"
    )

conn = p.connect(conn_str)

In [24]:
query="""
select distinct
ben.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num 
,ben.dep_fk 
,ben.comp_name
,mipst_dbo.tsf_ccm_template.template_label	
,mipst_dbo.tsf_ccm_template.template_description
,user_code
,replace(substring(call_start_datetime,1,10), '-','') as call_date 
,coalesce(g.comp_name,'none') as group_name	
,dep_type
,ben.comp_fk
,scheme_fk

FROM mipst_dbo.tsf_ccm_template
left join mipst_dbo.tsf_cct_call call ON call.template_obj = mipst_dbo.tsf_ccm_template.template_obj and call.status_key = 'cc_StCctcaComp' 
inner join mipst_dbo.tsd_cc_relationship real ON real.owning_obj::NUMERIC = call.call_obj
and UPPER(owning_entity_mnemonic) = 'CCTCA'
AND   UPPER(related_entity_mnemonic) in ('MEMBER','MEMDEP')
inner join  mipbi_dbo.td_beneficiary ben ON ben.ms_pk = coalesce(nullif(split_part(related_key, '|',1), ''), '0') --and scheme_fk='126'
and dep_fk::int	=coalesce(nullif(split_part(related_key, '|',2), ''), '0')::int
left join mipst_dbo.tsd_ben_memcom g  on  (case when ben.group_code='0' then ben.comp_fk else ben.group_code end)=g.comp_fk
"""

In [26]:
cte_df=pd.read_sql(query,conn)

In [28]:
eapraw_df['call_date_py']=pd.to_datetime(eapraw_df['call_date'], format='%Y-%m-%d', errors='ignore')

In [46]:
group_name_filter=eapraw_df['group_name']=='Capitec Bank Limited'
group_name_list=list(set(eapraw_df['group_name']))
group_name_list

['Merchants iiNet',
 'Pamodzi Unique Engineering',
 'LexisNexis (Pty) Ltd',
 'Bachique Group of Companies',
 'Hollard Life Assurance Company Limited',
 'Festo (Pty) Ltd',
 'Sanlam Individual',
 'WSP Group (Pty) Ltd',
 'KPMG Services (Pty) Limited',
 'IusPrudentia Specialist Counsel',
 'Freedom Technologies (Pty) Ltd',
 'Tau Phola (Pty) Ltd',
 'Agristar Group',
 'MSA DEVCO (Pty) Ltd trading as McDonalds SA',
 'HOSMED ESSENTIAL',
 'Rossmin Group',
 'Kevro (Pty) Ltd T/A Barron',
 'Twinsavers',
 'Calgro M3  Limited',
 'Beyond Zero',
 'Ocean View Fishing CC',
 'Iron Mountain SA',
 'Rhino Energy Solutions (Pty) Ltd',
 'Penguin Random House South Africa (Pty) Ltd',
 'SUREMED EXPLOR',
 'McDonalds - Auram Restaurants Company (Pty) Ltd',
 'Sysmex SA (Pty) Ltd',
 'Falcorp Technologies (Pty) Ltd',
 'Oceana Group',
 'FLEXICARE INDIVIDUAL MEMBERS',
 'Jones & Wagener (Pty) Ltd',
 'Herbalife International South Africa',
 'Enviroserv Waste Mngmt (Pty) Ltd',
 'Schindler Lifts (SA) (Pty) Ltd',
 'Focus Gr

In [42]:
# eapraw_df['call_date_py'].apply(lambda x: str(x.year)+f'{str(x.month):03}')
eapraw_df['call_date_my']=eapraw_df['call_date_py'].apply(lambda x: str(x.year)+str(x.month).zfill(2))
# n.zfill(3)

In [47]:
table = pd.pivot_table(eapraw_df[group_name_filter][['dep_type','call_date_my']],  index=['dep_type'],
                    columns=['call_date_my'], aggfunc=len)

In [48]:
table

call_date_my,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,...,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102
dep_type,,,,,,,,,,,,,,,,,,,,,
A,NaN,NaN,1.0,5.0,1.0,2.0,2.0,NaN,2.0,NaN,...,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,NaN
C,NaN,NaN,NaN,3.0,NaN,NaN,2.0,1.0,2.0,NaN,...,3.0,3.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN
M,48.0,209.0,456.0,532.0,676.0,532.0,885.0,664.0,746.0,736.0,...,525.0,738.0,874.0,844.0,977.0,1130.0,1156.0,824.0,865.0,572.0
O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
S,NaN,2.0,4.0,1.0,11.0,4.0,6.0,3.0,2.0,5.0,...,2.0,1.0,2.0,7.0,6.0,6.0,3.0,NaN,5.0,6.0


In [30]:
eapraw_df['call_date_py'].apply(lambda x: x.year)
# df['call_date_my_py']=df['call_date_py'].apply(lambda x: datetime(x.year,x.month,1))

0        2019
1        2019
2        2020
3        2020
4        2020
         ... 
94422    2019
94423    2019
94424    2019
94425    2019
94426    2019
Name: call_date_py, Length: 94427, dtype: int64